In [8]:
from langchain.agents import tool
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# get_word_length.invoke("abc")

tools = [TavilySearchResults(max_results=1)]

In [9]:
from modules.openai_agent import create_agent
from modules.create_graph import create_graph, stream_app
from modules.langraph_nodes import should_continue, call_model, call_tool, call_agent
from functools import partial

model = create_agent(tools=tools, model_only=True, streaming=True)

nodes = [{"name": "agent", "node": partial(call_model, model)}, 
         {"name": "action", "node": partial(call_tool, tools), "condition": should_continue}]

app = create_graph(nodes)

In [10]:
user_input = "what is the weather in sf"
stream_app(app, user_input)

Output from node 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_UDHhiaNfrtTnkPyUmUzVg6Bt', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]})]}

---

Output from node 'action':
---
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "Weather in San Francisco is {\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1708574779, \'localtime\': \'2024-02-21 20:06\'}, \'current\': {\'last_updated_epoch\': 1708574400, \'last_updated\': \'2024-02-21 20:00\', \'temp_c\': 12.8, \'temp_f\': 55.0, \'is_day\': 0, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/night/116.png\', \'code\': 1003}, \'wind_mph\': 8.1, \'wind_kph\': 13.0, \'wind_de